In [ ]:
#| default_exp core

In [ ]:
from dialoghelper import *

# dialoghelper

In [ ]:
#| export
import json,importlib,linecache,re,inspect,time,uuid,base64,sys
from io import BytesIO
import matplotlib.pyplot as plt
from typing import Dict
from tempfile import TemporaryDirectory
from ipykernel_helper import *
from dataclasses import dataclass
from os.path import normpath
from fastcore.xml import to_xml

from fastcore.utils import *
from fastcore.meta import delegates
from ghapi.all import *
from fastcore.xtras import asdict
from inspect import currentframe,Parameter,signature
from httpx import get as xget, post as xpost
from IPython.display import display,Markdown
from monsterui.all import franken_class_map,apply_classes
from fasthtml.common import Safe,Script,Div
from toolslm.xml import *
from IPython.core.ultratb import VerboseTB

In [ ]:
from fastcore import tools

## Helpers

In [ ]:
#| export
md_cls_d = {
    **{f'h{i}': f'uk-h{i}' for i in range(1,5)},
    'a': "uk-link",
    'blockquote': "uk-blockquote",
    'hr':'uk-divider-icon',
    'table':'uk-table uk-table-sm uk-table-middle uk-table-divider border [&_tr]:divide-x w-[80%] mx-auto',
    'ol': 'uk-list uk-list-decimal space-y-0', 
    'ul': 'uk-list uk-list-bullet space-y-0',
    'p': 'leading-tight',
    'li': 'leading-tight',
    'pre': '', 'pre code': '',
    'code': 'tracking-tight'
}

def add_styles(s:str, cls_map:dict=None):
    "Add solveit styles to `s`"
    return Safe(apply_classes(s, class_map=cls_map or md_cls_d))

In [ ]:
import mistletoe
from fasthtml.common import show

In [ ]:
s = mistletoe.markdown("### hi\n\n- first\n- *second*")
s

'<h3>hi</h3>\n<ul>\n<li>first</li>\n<li><em>second</em></li>\n</ul>\n'

In [ ]:
show(s)

In [ ]:
show(add_styles(s))

## Basics

In [ ]:
#| export
def _find_frame_dict(var:str):
    "Find the dict (globals or locals) containing var"
    frame = currentframe().f_back.f_back
    while frame:
        if var in frame.f_globals: return frame.f_globals
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

def find_var(var:str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]
    
def set_var(var:str, val):
    "Set var to val after finding it in all frames of the call stack"
    _find_frame_dict(var)[var] = val

In [ ]:
a = 1
find_var('a')

1

In [ ]:
set_var('a', 42)
a

42

In [ ]:
#| export
dh_settings = {'port':5001}

In [ ]:
# dh_settings = {'port':6001}

In [ ]:
#| export
def find_dname(dname=None):
    "Get the dialog name by searching the call stack for __dialog_id, and resolving `dname` if supplied."
    if dname:
        dname = dname.removesuffix('.ipynb')
        if dname.startswith('/'): return dname
    curr = dh_settings.get('dname', find_var('__dialog_name'))
    if not dname: return '/'+curr
    p = Path(curr).parent
    res = normpath((p/dname))
    assert '../' not in res, "Path traversal not permitted"
    return '/'+res

def find_msg_id():
    "Get the message id by searching the call stack for __msg_id."
    return find_var('__msg_id')

def _diff_dialog(pred, dname, err="`id` parameter must be provided when target dialog is different", id=None):
    "Raise ValueError if targeting a different dialog, `pred` is True, and no `id` provided"
    if not pred or id: return
    if dname or ('dname' in dh_settings): raise ValueError(err)

In [ ]:
find_dname()

'/aai-ws/dialoghelper/nbs/00_core'

In [ ]:
find_dname('index')

'/aai-ws/dialoghelper/nbs/index'

In [ ]:
find_dname('../index')

'/aai-ws/dialoghelper/index'

In [ ]:
find_dname('/foo/bar')

'/foo/bar'

In [ ]:
#| export
def call_endp(path, dname='', json=False, raiseex=False, id=None, **data):
    dname = find_dname(dname).strip('/')
    data['dlg_name'] = dname
    if id: data['id_'] = id
    headers = {'Accept': 'application/json'} if json else {}
    res = xpost(f'http://localhost:{dh_settings["port"]}/{path}', data=data, headers=headers)
    if raiseex: res.raise_for_status()
    try: return dict2obj(res.json()) if json else res.text
    except Exception as e: return str(e)

In [ ]:
find_msg_id()

'_9cbd170d'

In [ ]:
#| export
@llmtool
def curr_dialog(
    with_messages:bool=False,  # Include messages as well?
    dname:str='' # Dialog to get info for; defaults to current dialog
):
    "Get the current dialog info."
    res = call_endp('curr_dialog_', dname, json=True, with_messages=with_messages)
    if res: return {'name': res['name'], 'mode': res['mode']}

In [ ]:
#| export
@llmtool
def msg_idx(
    id:str=None,  # Message id to find (defaults to current message)
    dname:str='' # Dialog to get message index from; defaults to current dialog
):
    "Get absolute index of message in dialog."
    _diff_dialog(True, dname, id=id)
    if not id: id = find_msg_id()
    return call_endp('msg_idx_', dname, json=True, id=id)['idx']

In [ ]:
msg_idx()

27

In [ ]:
#| export
def add_scr(scr, oob='innerHTML:#ephemeral'):
    "Swap a script element to the end of the ephemeral element"
    if isinstance(scr,str): scr = Script(scr)
    add_html(Div(scr, hx_swap_oob=oob))

In [ ]:
#| export
def iife(code: str) -> str:
    "Wrap javascript code string in an IIFE and execute it via `add_html`"
    add_scr(f'''
(async () => {{
{code}
}})();
''')

In [ ]:
#| export
def pop_data(idx, timeout=15):
    return dict2obj(call_endp('pop_data_blocking_', data_id=idx, timeout=timeout, json=True))

In [ ]:
#| export
def fire_event(evt:str, data=None):
    params = f"'{evt}'"
    if data is not None: params += f", {json.dumps(data)}"
    add_html(Script(f"htmx.trigger(document.body, {params});", id='js-event', hx_swap_oob='true'))

In [ ]:
#| export
def event_get(evt:str, timeout=15, data=None):
    "Call `fire_event` and then `pop_data` to get a response"
    idx = uuid.uuid4()
    if not data: data = {}
    data['idx'] = str(idx)
    fire_event(evt, data=data)
    return pop_data(idx, timeout)

## View/edit dialog

In [ ]:
#| export
def _maybe_xml(res, as_xml, key=None):
    if as_xml: return res
    res = loads(res)
    if 'error' in res: return res
    if key: res = res[key]
    return dict2obj(res)

In [ ]:
#| export
@llmtool
def find_msgs(
    re_pattern:str='', # Optional regex to search for (re.DOTALL+re.MULTILINE is used)
    msg_type:str=None, # optional limit by message type ('code', 'note', or 'prompt')
    use_case:bool=False, # Use case-sensitive matching?
    use_regex:bool=True, # Use regex matching?
    only_err:bool=False, # Only return messages that have errors?
    only_chg:bool=False, # Only return messages that have changed vs git HEAD?
    ids:str='', # Optionally filter by comma-separated list of message ids
    limit:int=None, # Optionally limit number of returned items
    include_output:bool=True, # Include output in returned dict?
    include_meta:bool=True, # Include all additional message metadata
    as_xml:bool=False, # Use concise unescaped XML output format
    nums:bool=False, # Show line numbers?
    trunc_out:bool=False, # Middle-out truncate code output to 100 characters?
    trunc_in:bool=False, # Middle-out truncate cell content to 80 characters?
    headers_only:bool=False, # Only return note messages that are headers (first line only); cannot be used together with `header_section`
    header_section:str=None, # Find section starting with this header; returns it plus all children (i.e until next header of equal or more significant level)
    dname:str='' # Dialog to get info for; defaults to current dialog
)->list[dict]: # Messages in requested dialog that contain the given information
    """Often it is more efficient to call `view_dlg` to see the whole dialog at once, so you can use it all from then on, instead of using `find_msgs`.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension.
    Message ids are identical to those in LLM chat history, so do NOT call this to view a specific message if it's in the chat history--instead use `read_msgid`.
    Do NOT use find_msgs to view message content in the current dialog above the current prompt -- these are *already* provided in LLM context, so just read the content there directly. (NB: LLM context only includes messages *above* the current prompt, whereas `find_msgs` can access *all* messages.)
    To refer to a found message from code or tools, use its `id` field."""
    res = call_endp('find_msgs_', dname, json=False, re_pattern=re_pattern, msg_type=msg_type, limit=limit, ids=ids,
                    use_case=use_case, use_regex=use_regex, only_err=only_err, only_chg=only_chg,
                    include_output=include_output, include_meta=include_meta, as_xml=as_xml, nums=nums, trunc_out=trunc_out, trunc_in=trunc_in,
                    headers_only=headers_only, header_section=header_section)
    return _maybe_xml(res, as_xml=as_xml, key='msgs')

In [ ]:
# NB: must have a dialogue open including a message with this text in its content
txt = 'tools'
found = find_msgs(txt)

In [ ]:
found[0]['content']

'# Solveit Reference\n\n## What is Solveit?\n\nSolveit is a "Dialog Engineering" web application for interactive development. Unlike ChatGPT (pure chat) or Jupyter (pure code), Solveit combines three message types in one workspace: code execution, markdown notes, and AI prompts. Users build solutions incrementally—writing a few lines, understanding them, then continuing—rather than generating large code blocks.\n\nThe AI sees the full dialog context (code, outputs, notes, prompts) when responding -- but only those ABOVE the current message. Users can edit any message at any time, including AI responses—the dialog is a living document, not an append-only log.\n\nThe dialog is a running ipykernel instance. A "dialog" is like a "Jupyter notebook", and uses a compatible ipynb file format, but provides a superset of functionality (in particular, "prompt messages"). A "message" is like a "Jupyter cell", with additional attributes stored as ipynb cell metadata. Most standard jupyter functiona

In [ ]:
1+1

2

In [ ]:
r = find_msgs(r'1\+1', include_meta=False, include_output=True)
r

[{'id': '_8ce548d6', 'is_exported': 0, 'content': '1+1', 'output': '2', 'msg_type': 'code'}, {'id': '_44cb1b2a', 'is_exported': 0, 'content': "_id = _add_msg_unsafe('1+1', run=True, msg_type='code')", 'output': '', 'msg_type': 'code'}, {'id': '_6e354677', 'is_exported': 0, 'content': '1+1', 'output': '', 'msg_type': 'code'}]

In [ ]:
hl_md(find_msgs(r'1\+1', include_meta=False, as_xml=True))

```html
<msgs><code id="_8ce548d6"><source>1+1<out>2</out></code><code id="_44cb1b2a">_id = _add_msg_unsafe('1+1', run=True, msg_type='code')</code><code id="_6e354677">1+1</code></msgs>
```

In [ ]:
#| export
@llmtool
def view_dlg(
    dname:str='', # Dialog to get info for; defaults to current dialog
    msg_type:str=None, # optional limit by message type ('code', 'note', or 'prompt')
    nums:bool=False, # Whether to show line numbers
    include_output:bool=False, # Include output in returned dict?
    trunc_out:bool=True, # Middle-out truncate code output to 100 characters (only applies if `include_output`)?
    trunc_in:bool=False, # Middle-out truncate cell content to 80 characters?
):
    "Concise XML view of all messages (optionally filtered by type), not including metadata. Often it is more efficient to call this to see the whole dialog at once (including line numbers if needed), instead of running `find_msgs` or `read_msg` multiple times."
    return find_msgs(msg_type=msg_type, dname=dname, as_xml=True, nums=nums,
        include_meta=False, include_output=include_output, trunc_out=trunc_out, trunc_in=trunc_in)

In [ ]:
hl_md(view_dlg(nums=True)[:500])

```html
<msgs><code id="_955b9784">     1 │ #| default_exp core</code><note id="_3093ead7">     1 │ # Solveit Reference
     2 │ 
     3 │ ## What is Solveit?
     4 │ 
     5 │ Solveit is a "Dialog Engineering" web application for interactive development. Unlike ChatGPT (pure chat) or Jupyter (pure code), Solveit combines three message types in one workspace: code execution, markdown notes, and AI prompts. Users build solutions incrementally—writing a few lines, understanding them, then continuing—rath
```

In [ ]:
#| export
def add_html(
    content:str, # The HTML to send to the client (generally should include hx-swap-oob)
    dname:str='' # Dialog to get info for; defaults to current dialog
):
    "Send HTML to the browser to be swapped into the DOM"
    call_endp('add_html_', dname, content=to_xml(content))
    return {'success': 'Content added to DOM'}

In [ ]:
from fasthtml.common import *

In [ ]:
add_html(Div(P('Hi'), hx_swap_oob='beforeend:#dialog-container'))

{'success': 'Content added to DOM'}

In [ ]:
#| export
Placements = str_enum('Placements', 'add_after', 'add_before', 'at_start', 'at_end')

In [ ]:
#| export
@llmtool
def read_msg(
    n:int=-1,      # Message index (if relative, +ve is downwards)
    relative:bool=True,  # Is `n` relative to current message (True) or absolute (False)?
    id:str=None,  # Message id to find (defaults to current message)
    view_range:list[int,int]=None, # Optional 1-indexed (start, end) line range for files, end=-1 for EOF
    nums:bool=False, # Whether to show line numbers
    dname:str='' # Dialog to get info for; defaults to current dialog
    ):
    """Get the message indexed in the current dialog.
    NB: Messages in the current dialog above the current message are *already* visible; use this only when you need line numbers for editing operations, or for messages not in the current dialog or below the current message.
    - To get the exact message use `n=0` and `relative=True` together with `id`.
    - To get a relative message use `n` (relative position index).
    - To get the nth message use `n` with `relative=False`, e.g `n=0` first message, `n=-1` last message.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    _diff_dialog(relative, dname, "`id` parameter must be provided, or use `relative=False` with `n`, when target dialog is different", id=id)
    if relative and not id: id = find_msg_id()
    data = dict(n=n, relative=relative, id=id)
    if view_range: data['view_range'] = view_range # None gets converted to '' so we avoid passing it to use the p.default
    if nums: data['nums'] = nums
    return call_endp('read_msg_', dname, json=True, **data)

In [ ]:
#| export
@llmtool
def read_msgid(
    id:str,  # Message id to find
    view_range:list[int,int]=None, # Optional 1-indexed (start, end) line range for files, end=-1 for EOF
    nums:bool=False, # Whether to show line numbers
    dname:str='' # Dialog to get message from; defaults to current dialog
    ):
    """Get message `id`. Message IDs can be view directly in LLM chat history/context, or found in `find_msgs` results.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    return read_msg(0, id=id, view_range=view_range, nums=nums, dname=dname)

In [ ]:
#| export
@llmtool
def add_msg(
    content:str, # Content of the message (i.e the message prompt, code, or note text)
    placement:str='add_after', # Can be 'at_start' or 'at_end', and for default dname can also be 'add_after' or 'add_before'
    id:str=None, # id of message that placement is relative to (if None, uses current message; note: each add_msg updates "current" to the newly created message)
    msg_type: str='note', # Message type, can be 'code', 'note', or 'prompt'
    output:str='', # Prompt/code output; Code outputs must be .ipynb-compatible JSON array
    time_run: str | None = '', # When was message executed
    is_exported: int | None = 0, # Export message to a module?
    skipped: int | None = 0, # Hide message from prompt?
    i_collapsed: int | None = 0, # Collapse input?
    o_collapsed: int | None = 0, # Collapse output?
    heading_collapsed: int | None = 0, # Collapse heading section?
    pinned: int | None = 0, # Pin to context?
    dname:str='' # Dialog to get info for; defaults to current dialog. If passed, provide `id` or use `placement='at_start'`/`'at_end'`
):
    """Add/update a message to the queue to show after code execution completes.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    _diff_dialog(placement not in ('at_start','at_end'), dname,
        "`id` or `placement='at_end'`/`placement='at_start'` must be provided when target dialog is different", id=id)
    if placement not in ('at_start','at_end') and not id: id = find_msg_id()
    res = call_endp(
        'add_relative_', dname, content=content, placement=placement, id=id, msg_type=msg_type, output=output,
        time_run=time_run, is_exported=is_exported, skipped=skipped, pinned=pinned,
        i_collapsed=i_collapsed, o_collapsed=o_collapsed, heading_collapsed=heading_collapsed)
    set_var('__msg_id', res)
    return res

In [ ]:
print(__msg_id)
_id = add_msg('testing')
print(__msg_id)

_9c544573
_484f8955


In [ ]:
print(read_msg().content)

testing


`read_msg` (and all endpoints that return json) wrap responses in `dict2obj`, so you can use either dict or object syntax.

In [ ]:
bmsg = add_msg('at bottom', placement='at_end')

In [ ]:
assert(msg_idx(bmsg)>msg_idx(_id)+10)

In [ ]:
# dh_settings['dname'] = 'tmp'
# _id = add_msg('testing', placement='at_end')
# print(_id)
# del(dh_settings['dname'])

In [ ]:
#| export
@llmtool
def del_msg(
    id:str=None, # id of message to delete
    dname:str='' # Dialog to get info for; defaults to current dialog
):
    "Delete a message from the dialog. DO NOT USE THIS unless you have been explicitly instructed to delete messages."
    return call_endp('rm_msg_', dname, raiseex=True, msid=id, json=True)

In [ ]:
del_msg(bmsg)
del_msg(_id)

```python
{'status': 'success'}
```

In [ ]:
#| export
@delegates(add_msg)
def _add_msg_unsafe(
    content:str, # Content of the message (i.e the message prompt, code, or note text)
    placement:str='add_after', # Can be 'at_start' or 'at_end', and for default dname can also be 'add_after' or 'add_before'
    id:str=None, # id of message that placement is relative to (if None, uses current message)
    run:bool=False, # For prompts, send it to the AI; for code, execute it (*DANGEROUS -- be careful of what you run!)
    dname:str='', # Dialog to get info for; defaults to current dialog (`run` only has a effect if dialog is currently running)
    **kwargs
):
    """Add/update a message to the queue to show after code execution completes, and optionally run it.
    *WARNING*--This can execute arbitrary code, so check carefully what you run!--*WARNING"""
    _diff_dialog(placement not in ('at_start','at_end'), dname,
        "`id` or `placement='at_end'`/`placement='at_start'` must be provided when target dialog is different", id=id)    
    if placement not in ('at_start','at_end') and not id: id = find_msg_id()
    res = call_endp(
        'add_relative_', dname, content=content, placement=placement, id=id, run=run, **kwargs)
    set_var('__msg_id', res)
    return res

In [ ]:
_id = _add_msg_unsafe('1+1', run=True, msg_type='code')

In [ ]:
del_msg(_id)

```python
{'status': 'success'}
```

In [ ]:
_id = _add_msg_unsafe('Hi', run=True, msg_type='prompt')

In [ ]:
del_msg(_id)

```python
{'status': 'success'}
```

In [ ]:
#| export
class _StreamingWriter:
    """Captures stdout and streams updates to a solveit message."""
    def __init__(self, msg_id):
        self.msg_id, self.buffer, self.outputs = msg_id, "", []
    
    def write(self, text):
        self.buffer += text
        self._update()
        time.sleep(0.05)
    
    def flush(self): pass
    
    def _update(self):
        all_outputs = [{"output_type": "stream", "name": "stdout", "text": self.buffer}] if self.buffer else []
        all_outputs.extend(self.outputs)
        update_msg(id=self.msg_id, output=json.dumps(all_outputs))
    
    def add_image(self, b64):
        self.outputs.append({"output_type": "display_data", "data": {"image/png": b64}, "metadata": {}})
        self._update()
    
    def add_result(self, text):
        self.outputs.append({"output_type": "execute_result", "data": {"text/plain": text}, "metadata": {}, "execution_count": 1})
        self._update()
    
    def add_error(self, etype, evalue, tb):
        self.outputs.append({"output_type": "error", "ename": etype, "evalue": evalue, "traceback": tb})
        self._update()

In [ ]:
#| export
def _capture_figures():
    """Capture all matplotlib figures as base64 PNG images."""
    images = []
    if plt.get_fignums():
        for fig_num in plt.get_fignums():
            buf = BytesIO()
            plt.figure(fig_num).savefig(buf, format='png', bbox_inches='tight')
            buf.seek(0)
            images.append(base64.b64encode(buf.read()).decode())
        plt.close('all')
    return images

In [ ]:
#| export
def _exec_with_return(code):
    """Execute code and return value of last expression (if any)."""
    lines = code.strip().split('\n')
    if not lines: return None
    try:
        last = lines[-1]
        if lines[:-1]: exec('\n'.join(lines[:-1]), globals())
        return eval(last, globals())
    except SyntaxError:
        exec(code, globals())
        return None

In [ ]:
#| export
def _format_error(e):
    """Format exception with IPython-style colored traceback."""
    tb_formatter = VerboseTB(color_scheme='Linux')
    return {
        "etype": type(e).__name__,
        "evalue": str(e),
        "tb": tb_formatter.structured_traceback(type(e), e, sys.exc_info()[2])
    }

In [ ]:
#| export
@delegates(add_msg, but=['msg_type', 'output'])
def _exec_python(code: str, **kwargs):
    """Execute Python code with streaming output, matplotlib, errors, and return values."""
    msg_id = add_msg(code, msg_type='code', **kwargs)
    time.sleep(0.4)  # let oob_loop flush
    
    writer = _StreamingWriter(msg_id)
    old_stdout, sys.stdout = sys.stdout, writer
    result = None
    try:
        result = _exec_with_return(code)
        for img in _capture_figures(): writer.add_image(img)
        if result is not None: writer.add_result(repr(result))
    except Exception as e:
        err = _format_error(e)
        writer.add_error(err['etype'], err['evalue'], err['tb'])
    finally:
        sys.stdout = old_stdout
    
    return {'output': writer.buffer or "OK", 'id': msg_id, 'result': result}

In [ ]:
# _exec_python("2 + 2")

In [ ]:
# _exec_python("print('hello')")

In [ ]:
# _exec_python("1/0")  # test error handling

In [ ]:
#| export
def _umsg(
    content:str|None=None, # Content of the message (i.e the message prompt, code, or note text)
    msg_type: str|None = None, # Message type, can be 'code', 'note', or 'prompt'
    output:str|None = None, # Prompt/code output; Code outputs must be .ipynb-compatible JSON array
    time_run: str | None = None, # When was message executed
    is_exported: int | None = None, # Export message to a module?
    skipped: int | None = None, # Hide message from prompt?
    i_collapsed: int | None = None, # Collapse input?
    o_collapsed: int | None = None, # Collapse output?
    heading_collapsed: int | None = None, # Collapse heading section?
    pinned: int | None = None # Pin to context?
): ...

In [ ]:
#| export
@llmtool
@delegates(_umsg)
def update_msg(
    id:str=None, # id of message to update (if None, uses current message)
    msg:Optional[Dict]=None, # Dictionary of field keys/values to update
    dname:str='', # Dialog to get info for; defaults to current dialog
    **kwargs):
    """Update an existing message. Provide either `msg` OR field key/values to update.
    - Use `content` param to update contents.
    - Only include parameters to update--missing ones will be left unchanged.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    if msg: kwargs |= msg.get('msg', msg)
    if not id: id = kwargs.pop('id', None)
    if not id: raise TypeError("update_msg needs either a dict message with and id, or `id=`")
    res = call_endp('update_msg_', dname, id=id, **kwargs)
    set_var('__msg_id', res)
    return res

In [ ]:
_id = add_msg('testing')

In [ ]:
_id = update_msg(_id, content='toasting')

In [ ]:
_id = update_msg(_id, skipped=1)

In [ ]:
msg = read_msgid(_id)
msg['content'] = 'toasted'
update_msg(msg=msg)

'_e8ea44b9'

In [ ]:
del_msg(_id)

```python
{'status': 'success'}
```

In [ ]:
_edit_id = add_msg('This message should be found.\n\nThis is a multiline message.')

In [ ]:
print(read_msg()['content'])

This message should be found.

This is a multiline message.


In [ ]:
print(read_msg(n=0, id=_edit_id, nums=True)['content'])

     1 │ This message should be found.
     2 │ 
     3 │ This is a multiline message.


In [ ]:
print(read_msg(n=0, id=_edit_id, nums=True, view_range=[2,3])['content'])

     2 │ 
     3 │ This is a multiline message.


In [ ]:
#| export
def run_msg(
    ids:str=None, # Comma-separated ids of message(s) to execute
    dname:str='' # Running dialog to get info for; defaults to current dialog. (Note dialog *must* be running for this function)
):
    "Adds a message to the run queue. Use read_msg to see the output once it runs."
    return call_endp('add_runq_', dname, ids=ids, api=True)

In [ ]:
1+1

2

2

In [ ]:
codeid = read_msg()['id']

In [ ]:
run_msg(codeid)

'{"status":"queued"}'

In [ ]:
#| export
@llmtool
def copy_msg(
    ids:str=None, # Comma-separated ids of message(s) to copy
    cut:bool=False, # Cut message(s)? (If not, copies)
    dname:str='' # Running dialog to copy messages from; defaults to current dialog. (Note dialog *must* be running for this function)
):
    "Add `ids` to clipboard."
    id,*_ = ids.split(',')
    res = call_endp('msg_clipboard_', dname, ids=ids, id=id, cmd='cut' if cut else 'copy')
    return {'success':'complete'}

In [ ]:
#| export
@llmtool
def paste_msg(
    id:str=None, # Message id to paste next to
    after:bool=True, # Paste after id? (If not, pastes before)
    dname:str='' # Running dialog to copy messages from; defaults to current dialog. (Note dialog *must* be running for this function)
):
    "Paste clipboard msg(s) after/before the current selected msg (id)."
    res = call_endp('msg_paste_', dname, id=id, after=after)
    return {'success':'complete'}

In [ ]:
copy_msg(codeid)

{'success': 'complete'}

In [ ]:
tgt = read_msg()['id']

In [ ]:
paste_msg(tgt)

{'success': 'complete'}

In [ ]:
newmsg = read_msg(1, id=tgt)
newmsg['content']

'1+1'

In [ ]:
del_msg(newmsg['id'])

```python
{'status': 'success'}
```

In [ ]:
#| export
mermaid_url = "https://cdn.jsdelivr.net/npm/mermaid@11/dist/mermaid.esm.min.mjs"

def enable_mermaid():
    return Script("""
if (window.mermaid) mermaid.run()
else {
    import('%s').then(m => {
        window.mermaid = m.default;
        window.mermaid.run();
        htmx.onLoad(elt => {
            if (elt.matches('div.mermaid, pre.mermaid') || htmx.findAll(elt, 'div.mermaid, pre.mermaid')) window.mermaid.run();
        });
    });
}""" % mermaid_url, type="module")

In [ ]:
enable_mermaid()

<script type="module">
if (window.mermaid) mermaid.run()
else {
    import('https://cdn.jsdelivr.net/npm/mermaid@11/dist/mermaid.esm.min.mjs').then(m => {
        window.mermaid = m.default;
        window.mermaid.run();
        htmx.onLoad(elt => {
            if (elt.matches('div.mermaid, pre.mermaid') || htmx.findAll(elt, 'div.mermaid, pre.mermaid')) window.mermaid.run();
        });
    });
}</script>

In [ ]:
#| export
def mermaid(code, cls="mermaid", **kwargs):
    "A mermaid diagram"
    return Div(code, cls=cls, **kwargs)

In [ ]:
mermaid('graph LR; A[Start] --> B[Process]; B --> C[End];')

<div class="mermaid">graph LR; A[Start] --&gt; B[Process]; B --&gt; C[End];</div>

You can also add to a note:

    ```mermaid
    graph LR
    A[Start] --> B[Process]
    B --> C[End]
    ```

This renders as: 

```mermaid
graph LR
A[Start] --> B[Process]
B --> C[End]
```

You can also add to a note:

    ```mermaid
    graph LR
    A[Start] --> B[Process]
    B --> C[End]
    ```

This renders as: 

```
graph LR
A[Start] --> B[Process]
B --> C[End]
```

In [ ]:
#| export
@llmtool
def toggle_header(
    id:str, # id of markdown header note message to toggle collapsed state
    dname:str='' # Running dialog to copy messages from; defaults to current dialog. (Note dialog *must* be running for this function)
):
    "Toggle collapsed header state for `id`"
    res = call_endp('toggle_header_collapse_', dname, id=id)
    return {'success':'complete'}

#### test header

In [ ]:
hdid = read_msg()['id']

test note

#### header end

In [ ]:
toggle_header(hdid)

{'success': 'complete'}

In [ ]:
#| export
def url2note(
    url:str, # URL to read
    extract_section:bool=True, # If url has an anchor, return only that section
    selector:str=None, # Select section(s) using BeautifulSoup.select (overrides extract_section)
    ai_img:bool=True, # Make images visible to the AI
    split_re:str='' # Regex to split content into multiple notes, set to '' for single note
):
    "Read URL as markdown, and add note(s) below current message with the result"
    res = read_url(url, as_md=True, extract_section=extract_section, selector=selector, ai_img=ai_img)
    if split_re: return [add_msg(s) for s in re.split(split_re, res, flags=re.MULTILINE) if s.strip()]
    return add_msg(res)


In [ ]:
_id = url2note('https://www.example.org')

In [ ]:
del_msg(_id)

```python
{'status': 'success'}
```

In [ ]:
#| export
@llmtool
def create_dialog(
    name:str, # Name/path of the new dialog (relative to current dialog's folder, or absolute if starts with '/')
):
    "Create a new dialog"
    name = find_dname(name).lstrip('/')
    return call_endp('create_dialog_', name=name, api=True, json=True)

In [ ]:
create_dialog('test_dialog')

```python
{'success': '"aai-ws/dialoghelper/nbs/test_dialog" is now running'}
```

In [ ]:
#| export
def rm_dialog(
    name:str, # Name/path of the dialog to delete (relative to current dialog's folder, or absolute if starts with '/')
):
    "Delete a dialog (or folder) and associated records"
    name = find_dname(name).lstrip('/')
    return call_endp('rm_dialog_', name=name, sess='{}', api=True, json=True)

In [ ]:
rm_dialog('test_dialog')

```python
{'success': 'deleted "/Users/jhoward/aai-ws/dialoghelper/nbs/test_dialog"'}
```

In [ ]:
#| export
@llmtool
def run_code_interactive(
    code:str # Code to have user run
):
    """Insert code into user's dialog and request for the user to run it. Use other tools where possible, 
    but if they can not find needed information, *ALWAYS* use this instead of guessing or giving up.
    IMPORTANT: This tool is TERMINAL - after calling it, you MUST stop all tool usage 
    and wait for user response. Never call additional tools after this one."""
    add_msg('# Please run this:\n'+code, msg_type='code')
    return {'success': "CRITICAL: Message added to user dialog. STOP IMMEDIATELY. Do NOT call any more tools. Wait for user to run code and respond."}

In [ ]:
#| export
def dialog_link(
    path:str # Path to dialog (e.g. '/aai-ws/dialoghelper/nbs/00_core')
):
    """Return an IPython HTML link to open a dialog in Solveit.
    After calling this tool, output the resulting HTML anchor tag exactly as returned—do not wrap in a fenced code block or convert to markdown link format."""
    path = path.removeprefix('/')
    url = f"/dialog_?{urlencode({'name': path})}"
    return HTML(f'<a href="{url}" target="_blank">{path}</a>')

## Text Edit

In [ ]:
#| export
@llmtool
def msg_insert_line(
    id:str,  # Message id to edit
    insert_line: int, # The 1-based line number after which to insert the text (0: before 1st line, 1: after 1st line, 2: after 2nd, etc.)
    new_str: str, # The text to insert
    dname:str='', # Dialog to get info for; defaults to current dialog
    update_output:bool=False # If True, insert into output instead of content
):
    """Insert text at a specific line number in a message. Be sure you've called `read_msg(..., nums=True)` to ensure you know the line numbers.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    msg = read_msg(n=0, id=id, dname=dname)
    field = 'output' if update_output else 'content'
    text = msg.get(field, '')
    if not text: return {'error': f"Message has no {field}"}
    lines = text.splitlines()
    if not (0 <= insert_line <= len(lines)): return {'error': f'Invalid line number {insert_line}. Valid range: 0-{len(lines)}'}
    lines.insert(insert_line, new_str)
    update_msg(id=id, **{field: '\n'.join(lines)}, dname=dname)
    return {'success': f'Inserted text after line {insert_line} in message {id} {field}'}

In [ ]:
msg_insert_line(_edit_id, 0, 'This should go to the first line')
msg_insert_line(_edit_id, 3, 'This should go to the 4th line')
msg_insert_line(_edit_id, 5, 'This should go to the last line')

{'success': 'Inserted text after line 5 in message _4e05e042'}

In [ ]:
print(read_msg(n=0, id=_edit_id, nums=True)['content'])

     1 │ This should go to the first line
     2 │ This message should be found.
     3 │ 
     4 │ This should go to the 4th line
     5 │ This is a multiline message.
     6 │ This should go to the last line


In [ ]:
#| export
@llmtool
def msg_str_replace(
    id:str,  # Message id to edit
    old_str: str, # Text to find and replace
    new_str: str, # Text to replace with
    dname:str='', # Dialog to get info for; defaults to current dialog
    update_output:bool=False # If True, replace in output instead of content
):
    """Replace first occurrence of old_str with new_str in a message.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    msg = read_msg(n=0, id=id, dname=dname)
    field = 'output' if update_output else 'content'
    text = msg.get(field, '')
    if not text: return {'error': f"Message has no {field}"}
    count = text.count(old_str)
    if count == 0: return {'error': f"Text not found in message {field}: {repr(old_str)}"}
    if count > 1: return {'error': f"Multiple matches found ({count}) for text: {repr(old_str)}"}
    update_msg(id=id, **{field: text.replace(old_str, new_str, 1)}, dname=dname)
    return {'success': f'Replaced text in message {id} {field}'}

In [ ]:
msg_str_replace(_edit_id, 'This should go to the first line', 'This should go to the 1st line')

{'success': 'Replaced text in message _4e05e042'}

In [ ]:
print(read_msg(n=0, id=_edit_id, nums=True)['content'])

     1 │ This should go to the 1st line
     2 │ This message should be found.
     3 │ 
     4 │ This should go to the 4th line
     5 │ This is a multiline message.
     6 │ This should go to the last line


In [ ]:
#| export
@llmtool
def msg_strs_replace(
    id:str,  # Message id to edit
    old_strs:list[str], # List of strings to find and replace
    new_strs:list[str], # List of replacement strings (must match length of old_strs)
    dname:str='', # Dialog to get info for; defaults to current dialog
    update_output:bool=False # If True, replace in output instead of content
):
    """Replace multiple strings simultaneously in a message.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    if not isinstance(old_strs, list): return {'error': f"`old_strs` should be a list[str] but got {type(old_strs)}"}
    if not isinstance(new_strs, list): return {'error': f"`new_strs` should be a list[str] but got {type(new_strs)}"}
    if len(old_strs) != len(new_strs): return {'error': f"Length mismatch: {len(old_strs)} old_strs vs {len(new_strs)} new_strs"}
    msg = read_msg(n=0, id=id, dname=dname)
    field = 'output' if update_output else 'content'
    text = msg.get(field, '')
    if not text: return {'error': f"Message has no {field}"}
    for idx, (old_str, new_str) in enumerate(zip(old_strs, new_strs)):
        count = text.count(old_str)
        if count == 0: return {'error': f"Text not found in message {field} at index {idx}: {repr(old_str)}"}
        if count > 1: return {'error': f"Multiple matches ({count}) found at index {idx} for: {repr(old_str)}"}
        text = text.replace(old_str, new_str, 1)
    update_msg(id=id, **{field: text}, dname=dname)
    return {'success': f'Successfully replaced all the strings in message {id} {field}'}

In [ ]:
msg_strs_replace(_edit_id, ['This is a multiline message.', 'This should go to the last line'], ['5th line', 'last line'])

{'success': 'Successfully replaced all the strings in message _4e05e042'}

In [ ]:
print(read_msg(n=0, id=_edit_id, nums=True)['content'])

     1 │ This should go to the 1st line
     2 │ This message should be found.
     3 │ 
     4 │ This should go to the 4th line
     5 │ 5th line
     6 │ last line


In [ ]:
#| export
def _norm_lines(n:int, start:int, end:int=None):
    "Normalize and validate line range. Returns (start, end) or error dict."
    if end is None: end = start
    if end < 0: end = n + end + 1
    if not (1 <= start <= n): return {'error': f'Invalid start line {start}. Valid range: 1-{n}'}
    if not (start <= end <= n): return {'error': f'Invalid end line {end}. Valid range: {start}-{n}'}
    return start, end

In [ ]:
#| export
@llmtool
def msg_replace_lines(
    id:str,  # Message id to edit
    start_line:int, # Starting line number to replace (1-based indexing)
    end_line:int=None, # Ending line number to replace (1-based, inclusive, negative counts from end, None for single line)
    new_content:str='', # New content to replace the specified lines
    dname:str='', # Dialog to get info for; defaults to current dialog
    update_output:bool=False # If True, replace in output instead of content
):
    """Replace a range of lines with new content in a message.
    Be sure you've called `read_msg(..., nums=True)` to ensure you know the line numbers.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    msg = read_msg(n=0, id=id, dname=dname)
    field = 'output' if update_output else 'content'
    text = msg.get(field, '')
    if not text: return {'error': f"Message has no {field}"}
    lines = text.splitlines(keepends=True)
    res = _norm_lines(len(lines), start_line, end_line)
    if isinstance(res, dict): return res
    start_line, end_line = res
    if lines and new_content and not new_content.endswith('\n'): new_content += '\n'
    lines[start_line-1:end_line] = [new_content] if new_content else []
    update_msg(id=id, **{field: ''.join(lines)}, dname=dname)
    return {'success': f'Replaced lines {start_line} to {end_line} in message {id} {field}'}

In [ ]:
msg_replace_lines(_edit_id, 2, 4,'line 2\nline 3\nline 4\n')

{'success': 'Replaced lines 2 to 4 in message _4e05e042'}

In [ ]:
print(read_msg(n=0, id=_edit_id, nums=True)['content'])

     1 │ This should go to the 1st line
     2 │ line 2
     3 │ line 3
     4 │ line 4
     5 │ 5th line
     6 │ last line


In [ ]:
#| export
@llmtool
def msg_del_lines(
    id:str,  # Message id to edit
    start_line:int, # Starting line number to delete (1-based indexing)
    end_line:int=None, # Ending line number to delete (1-based, inclusive, negative counts from end, None for single line)
    dname:str='', # Dialog to get info for; defaults to current dialog
    update_output:bool=False # If True, delete from output instead of content
):
    """Delete a range of lines from a message. Be sure you've called `read_msg(..., nums=True)` to ensure you know the line numbers.
    If `dname` is None, the current dialog is used. If it is an open dialog, it will be updated interactively with real-time updates to the browser. If it is a closed dialog, it will be updated on disk. Dialog names must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    msg = read_msg(n=0, id=id, dname=dname)
    field = 'output' if update_output else 'content'
    text = msg.get(field, '')
    if not text: return {'error': f"Message has no {field}"}
    lines = text.splitlines(keepends=True)
    res = _norm_lines(len(lines), start_line, end_line)
    if isinstance(res, dict): return res
    start_line, end_line = res
    del lines[start_line-1:end_line]
    update_msg(id=id, **{field: ''.join(lines)}, dname=dname)
    return {'success': f'Deleted lines {start_line} to {end_line} in message {id} {field}'}

In [ ]:
msg_del_lines(_edit_id, 2, 4)

{'success': 'Deleted lines 2 to 4 in message _4e05e042'}

In [ ]:
print(read_msg(n=0, id=_edit_id, nums=True)['content'])

     1 │ This should go to the 1st line
     2 │ 5th line
     3 │ last line


In [ ]:
del_msg(_edit_id)

```python
{'status': 'success'}
```

In [ ]:
#| export
@llmtool
def dialoghelper_explain_dialog_editing(
)->str: # Detailed documention on dialoghelper dialog editing
    "Call this to get a detailed explanation of how dialog editing is done in dialoghelper. Always use if doing anything non-trivial, or if dialog editing has not previously occured in this session"
    return """# dialoghelper dialog editing functionality

This guide consolidates understanding of how dialoghelper tools work together. Individual tool schemas are already in context—this adds architectural insight and usage patterns.

## Core Concepts

**Dialog addressing**: All functions accepting `dname` resolve paths relative to current dialog (no leading `/`) or absolute from Solveit's runtime data path (with leading `/`). The `.ipynb` extension is never included.

**Message addressing**: Messages have stable `id` strings (e.g., `_a9cb5512`). The current executing message's id is in `__msg_id`. Tools use `id` for targeting; `find_msg_id()` retrieves current.

**Implicit state**: After `add_msg`/`update_msg`, `__msg_id` is updated to the new/modified message. This enables chaining: successive `add_msg` calls create messages in sequence.

## Tool Workflow Patterns

### Reading dialog state
- `view_dlg` — fastest way to see entire dialog structure with line numbers for editing
- `find_msgs` — search with regex, filter by type/errors/changes
- `read_msg` — navigate relative to current message
- `read_msgid` — direct access when you have the id

**Key insight**: Messages above the current prompt are already in LLM context. Use read tools only for: (1) getting line numbers for edits, (2) accessing messages below current prompt, (3) accessing other dialogs.

### Modifying dialogs
- `add_msg` — placement can be `add_after`/`add_before` (relative to current) or `at_start`/`at_end` (absolute)
- `update_msg` — partial updates; only pass fields to change
- `del_msg` — use sparingly, only when explicitly requested
`copy_msg` → `paste_msg` — for moving/duplicating messages within running dialogs.

## Non-decorated Functions Worth Knowing

There are additional functions available that can be added to fenced blocks, or the user may add as tools; they are not included in schemas by default.

**Browser integration:**
- `add_html(content)` — inject HTML with `hx-swap-oob` into live browser DOM
- `iife(code)` — execute JavaScript immediately in browser
- `fire_event(evt, data)` / `event_get(evt)` — trigger/await browser events

**Content helpers:**
- `url2note(url, ...)` — fetch URL as markdown, add as note message
- `mermaid(code)` / `enable_mermaid()` — render mermaid diagrams
- `add_styles(s)` — apply solveit's MonsterUI styling to HTML

**Dangerous (not exposed by default):**
- `_add_msg_unsafe(content, run=True, ...)` — add AND execute message (code or prompt)
- `run_msg(ids)` — queue messages for execution
- `rm_dialog(name)` — delete entire dialog

## Important Patterns

### Key Principles

1. **Always re-read before editing.** Past tool call results in chat history are TRUNCATED. Never rely on line numbers from earlier in the conversation—call `read_msgid(id, nums=True)` immediately before any edit operation.

2. **Work backwards.** When making multiple edits to a message, start from the end and work towards the beginning. This prevents line number shifts from invalidating your planned edits.

3. **Don't guess when tools fail.** If a tool call returns an error, STOP and ask for clarification. Do not retry with guessed parameters.

4. **Verify after complex edits.** After significant changes, re-read the affected region to confirm the edit worked as expected before proceeding.

### Typical Workflow

```
1. read_msgid(id, nums=True)           # Get current state with line numbers
2. Identify lines to change
3. msg_replace_lines(...) or msg_str_replace(...)  # Make edit
4. If more edits needed: re-read, then repeat from step 2
```

### Tool Selection

- **`msg_replace_lines`**: Best for replacing/inserting contiguous blocks. Use `view_range` on read to focus on the area.
- **`msg_str_replace`**: Best for targeted single-string replacements when you know the exact text.
- **`msg_strs_replace`**: Best for multiple independent replacements in one call.
- **`msg_insert_line`**: Best for adding new content without replacing existing lines.
- **`msg_del_lines`**: Best for removing content.

### Common Mistakes to Avoid

- Using line numbers from a truncated earlier result
- Making multiple edits without re-reading between them
- Guessing line numbers when a view_range was truncated
- Always call `read_msgid(id, nums=True)` first to get accurate line numbers
- String-based tools (`msg_str_replace`, `msg_strs_replace`) fail if the search string appears zero or multiple times—use exact unique substrings."""

## ast-grep

In [ ]:
#| export
def ast_py(code:str):
    "Get an SgRoot root node for python `code`"
    from ast_grep_py import SgRoot
    return SgRoot(code, "python").root()

In [ ]:
node = ast_py("print('hello world')")
stmt = node.find(pattern="print($A)")
res = stmt.get_match('A')
res.text(),res.range()

("'hello world'",
 Range(start=Pos(line=0, col=6, index=6), end=Pos(line=0, col=19, index=19)))

In [ ]:
#| export
@llmtool
def ast_grep(
    pattern:str, # ast-grep pattern to search, e.g "post($A, data=$B, $$$)"
    path:str=".", # path to recursively search for files
    lang:str="python" # language to search/scan
): # json format from calling `ast-grep --json=compact
    """Use `ast-grep` to find code patterns by AST structure (not text).
    
    Pattern syntax:
    - $VAR captures single nodes, $$$ captures multiple
    - Match structure directly: `def $FUNC($$$)` finds any function; `class $CLASS` finds classes regardless of inheritance
    - DON'T include `:` - it's concrete syntax, not AST structure
    - Whitespace/formatting ignored - matches structural equivalence
    
    Examples: `import $MODULE` (find imports); `$OBJ.$METHOD($$$)` (find method calls); `await $EXPR` (find await expressions)
    
    Useful for: Refactoring—find all uses of deprecated APIs or changed signatures; Security review—locate SQL queries, file operations, eval calls; Code exploration—understand how libraries are used across codebase; Pattern analysis—find async functions, error handlers, decorators; Better than regex—handles multi-line code, nested structures, respects syntax"""
    import json, subprocess
    cmd = f"ast-grep --pattern '{pattern}' --lang {lang} --json=compact"
    if path != ".": cmd = f"cd {path} && {cmd}"
    res = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return json.loads(res.stdout) if res.stdout else res.stderr

The `ast_grep` function calls the `ast-grep` CLI, which is a tool for searching code based on its structure rather than just text patterns. Unlike regular expressions that match character sequences, `ast-grep` understands the syntax of programming languages and lets you search for code patterns in a way that respects the language's grammar. This means you can find function calls, variable assignments, or other code constructs even when they're formatted differently or have varying amounts of whitespace.

The key advantage is using metavariables (like `$A`, `$B`, `$$$`) as placeholders in your search patterns. When you search for `xpost($A, data=$B, $$$)`, you're asking to find all calls to `xpost` where the first argument can be anything (captured as `$A`), there's a keyword argument `data` with any value (captured as `$B`), and there may be additional arguments after that (the `$$$` matches zero or more remaining arguments). This is much more reliable than trying to write a regex that handles all the variations of how that function might be called.

In the example below, we search for calls to `xpost` in the parent directory and extract both the matched code and the specific values of our metavariables, showing us exactly where and how this function is being used in the codebase.

In [ ]:
res = ast_grep(r"xpost($A, data=$B, $$$)", '..')
[(o['text'],o['metaVariables']['single'],o['file']) for o in res]

[('xpost(f\'http://localhost:{dh_settings["port"]}/{path}\', data=data, headers=headers)',
  {'A': {'text': 'f\'http://localhost:{dh_settings["port"]}/{path}\'',
    'range': {'byteOffset': {'start': 4081, 'end': 4129},
     'start': {'line': 99, 'column': 16},
     'end': {'line': 99, 'column': 64}}},
   'B': {'text': 'data',
    'range': {'byteOffset': {'start': 4136, 'end': 4140},
     'start': {'line': 99, 'column': 71},
     'end': {'line': 99, 'column': 75}}}},
  'dialoghelper/core.py')]

**Basic Patterns:**
- Match code structure directly: `console.log($ARG)` 
- Metavariables capture parts: `$VAR` (single), `$$$` (multiple)
- Patterns match AST structure, not text - whitespace/formatting doesn't matter

**The Colon Issue:**
- **Don't include `:` in patterns** - it's part of Python's concrete syntax, not the AST structure
- ✅ `def $FUNC($$$)` - matches function definitions
- ❌ `def $FUNC($$$):` - too specific, looking for the colon token itself

**When to use `kind` vs `pattern`:**
- `pattern`: Simple direct matches (`await $EXPR`)
- `kind`: Structural node types (`kind: function_declaration`)

**Critical rule for relational searches:**
Always add `stopBy: end` to `has`/`inside` rules to search the entire subtree:
```yaml
has:
  pattern: await $EXPR
  stopBy: end
```

**Escaping in shell:**
Use `\$VAR` or single quotes when using `--inline-rules` from command line

## Context

In [ ]:
#| export
@delegates(folder2ctx)
def ctx_folder(
    path:Path='.',  # Path to collect
    types:str|list='py,doc',  # list or comma-separated str of ext types from: py, js, java, c, cpp, rb, r, ex, sh, web, doc, cfg
    out=False, # Include notebook cell outputs?
    raw=True, # Add raw message, or note?
    exts:str|list=None, # list or comma-separated str of exts to include (overrides `types`)
    **kwargs
):
    "Convert folder to XML context and place in a new message"
    if exts: types=None
    res = folder2ctx(path, types=types, out=out, exts=exts, **kwargs)
    if not raw: res = f'```\n{res}\n```'
    return add_msg(res, msg_type='raw' if raw else 'note')

In [ ]:
# ctx_folder('..', max_total=600, sigs_only=True, exts='py')

In [ ]:
#| export
@delegates(repo2ctx)
def ctx_repo(
    owner:str,  # GitHub repo owner
    repo:str,   # GitHub repo name
    types:str|list='py,doc',  # list or comma-separated str of ext types from: py, js, java, c, cpp, rb, r, ex, sh, web, doc, cfg
    exts:str|list=None, # list or comma-separated str of exts to include (overrides `types`)
    out=False, # Include notebook cell outputs?
    raw=True, # Add raw message, or note?
    **kwargs
):
    "Convert GitHub repo to XML context and place in a new message"
    res = repo2ctx(owner, repo, out=out, types=types, exts=exts, **kwargs)
    if exts: types=None
    if not raw: res = f'```\n{res}\n```'
    return add_msg(res, msg_type='raw' if raw else 'note')

In [ ]:
#| export
def ctx_symfile(sym):
    "Add note with filepath and contents for a symbol's source file"
    return add_msg(sym2file(sym), msg_type='note');

In [ ]:
# ctx_symfile(TemporaryDirectory)

In [ ]:
#| export
@delegates(sym2folderctx)
def ctx_symfolder(
    sym, # Symbol to get folder context from
    **kwargs):
    "Add raw message with folder context for a symbol's source file location"
    return add_msg(sym2folderctx(sym, **kwargs), msg_type='raw');

In [ ]:
# ctx_symfolder(folder2ctx)

In [ ]:
#| export
@delegates(sym2folderctx)
def ctx_sympkg(
    sym, # Symbol to get folder context from
    **kwargs):
    "Add raw message with repo context for a symbol's root package"
    return add_msg(sym2pkgctx(sym, **kwargs), msg_type='raw');

In [ ]:
# ctx_sympkg(folder2ctx)

## Gists

In [ ]:
#| export
def load_gist(gist_id:str):
    "Retrieve a gist"
    api = GhApi()
    if '/' in gist_id: *_,user,gist_id = gist_id.split('/')
    else: user = None
    return api.gists.get(gist_id, user=user)

In [ ]:
gistid = 'jph00/e7cfd4ded593e8ef6217e78a0131960c'
gist = load_gist(gistid)
gist.html_url

'https://gist.github.com/jph00/e7cfd4ded593e8ef6217e78a0131960c'

In [ ]:
#| export
def gist_file(gist_id:str):
    "Get the first file from a gist"
    gist = load_gist(gist_id)
    return first(gist.files.values())

In [ ]:
gfile = gist_file(gistid)
print(gfile.content[:100]+"…")

"This is a test module which makes some simple tools available."
__all__ = ["hi","whoami"]

testfoo=…


In [ ]:
#| export
def import_string(
    code:str, # Code to import as a module
    name:str  # Name of module to create
):
    with TemporaryDirectory() as tmpdir:
        path = Path(tmpdir) / f"{name}.py"
        path.write_text(code)
        # linecache.cache storage allows inspect.getsource() after tmpdir lifetime ends
        linecache.cache[str(path)] = (len(code), None, code.splitlines(keepends=True), str(path))
        spec = importlib.util.spec_from_file_location(name, path)
        module = importlib.util.module_from_spec(spec)
        sys.modules[name] = module
        spec.loader.exec_module(module)
        return module

In [ ]:
def hi(who:str):
    "Say hi to `who`"
    return f"Hello {who}"

def hi2(who):
    "Say hi to `who`"
    return f"Hello {who}"

def hi3(who:str):
    return f"Hello {who}"

bye = "bye"

In [ ]:
assert is_usable_tool(hi)
assert not is_usable_tool(hi2)
assert not is_usable_tool(hi3)
assert not is_usable_tool(bye)

In [ ]:
#| export
def mk_toollist(syms):
    return "\n".join(f"- &`{sym.__name__}`: {sym.__doc__}" for sym in syms if is_usable_tool(sym))

In [ ]:
Markdown(mk_toollist([hi]))

- &`hi`: Say hi to `who`

In [ ]:
#| export
def import_gist(
    gist_id:str, # user/id or just id of gist to import as a module
    mod_name:str=None, # module name to create (taken from gist filename if not passed)
    add_global:bool=True, # add module to caller's globals?
    import_wildcard:bool=False, # import all exported symbols to caller's globals
    create_msg:bool=False # Add a message that lists usable tools
):
    "Import gist directly from string without saving to disk"
    fil = gist_file(gist_id)
    mod_name = mod_name or Path(fil['filename']).stem
    module = import_string(fil['content'], mod_name)
    glbs = currentframe().f_back.f_globals
    if add_global: glbs[mod_name] = module
    syms = getattr(module, '__all__', None)
    if syms is None: syms = [o for o in dir(module) if not o.startswith('_')]
    syms = [getattr(module, nm) for nm in syms]
    if import_wildcard:
        for sym in syms: glbs[sym.__name__] = sym
    if create_msg:
        pref = getattr(module, '__doc__', "Tools added to dialog:")
        add_msg(f"{pref}\n\n{mk_toollist(syms)}")
    return module

In [ ]:
import_gist(gistid)
importtest.testfoo

'testbar'

In [ ]:
import_gist.__doc__

'Import gist directly from string without saving to disk'

In [ ]:
import_gist(gistid, import_wildcard=True)
importtest.testfoo

'testbar'

In [ ]:
hi("Sarah")

'Hello Sarah'

In [ ]:
importtest.__all__

['hi', 'whoami']

In [ ]:
#| export
def update_gist(gist_id:str, content:str):
    "Update the first file in a gist with new content"
    api = GhApi()
    if '/' in gist_id: *_,user,gist_id = gist_id.split('/')
    gist = api.gists.get(gist_id)
    fname = first(gist.files.keys())
    res = api.gists.update(gist_id, files={fname: {'content': content}})
    return res['html_url']

## export -

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()